In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Scripts

Any alias that is a composition of two or more commands.

In [4]:
%%sql
with total as (select count(*) as total from alias)
select 
    count(distinct alias_id) as '#', 
    round(count(distinct alias_id)*100.0/total,2) as '%' 
from alias join command using (alias_id)
join total
where alias.num_commands != 1;


 * sqlite:///results.db
Done.


#,%
204142,9.26


In [6]:
%%sql
with total as (select count(*) as total from alias)
select num_commands, count(*) as '#', round(count(*)*100.0/total,2) as '%'
from alias 
join total
group by num_commands
order by count(*) desc
limit 10;

 * sqlite:///results.db
Done.


num_commands,#,%
1,2000057,90.74
2,137740,6.25
3,38178,1.73
4,15095,0.68
5,5799,0.26
6,3471,0.16
7,1481,0.07
8,998,0.05
9,642,0.03
10,263,0.01


### Most used operators

In [11]:
%%sql
with total as (select count(*) as total from command where operator is not null)
select operator, count(*), round(count(*)*100.0/total, 2) 
from command 
join total
where operator is not null
group by operator 
order by 2 desc

 * sqlite:///results.db
Done.


operator,count(*),"round(count(*)*100.0/total, 2)"
|,137000,39.66
;,102277,29.61
&&,92849,26.88
&,9653,2.79
||,2910,0.84
|&,746,0.22


### Operators by position

In [19]:
%%sql ops_pos << 
select operator, position as pos, count(*) as num 
from command
where operator is not null
group by position, operator 
order by operator, position asc, num desc

 * sqlite:///results.db
Done.
Returning data to local variable ops_pos


In [20]:
ops_pos = ops_pos.DataFrame().pivot(index='pos', columns='operator', values='num')[:11]
ops_pos = ops_pos.sort_values(ops_pos.first_valid_index(), axis=1, ascending=False)
ops_pos

operator,|,|&,&,;,&&,||
pos,,,,,,
0,14619.0,671.0,62.0,61.0,51.0,3.0
1,78082.0,62.0,8109.0,53208.0,62901.0,1780.0
2,25622.0,10.0,897.0,22083.0,17124.0,666.0
3,9923.0,3.0,249.0,11100.0,6744.0,205.0
4,4307.0,NaN,102.0,5781.0,2824.0,115.0
5,2617.0,NaN,54.0,3370.0,1247.0,42.0
6,903.0,NaN,33.0,2215.0,686.0,22.0
7,464.0,NaN,29.0,1502.0,368.0,15.0
8,149.0,NaN,22.0,985.0,215.0,9.0
